# Build Advisor PoC

AI Applications

- Audience is technical
- LLM + good prompt engineering

Action Item:
- Work with Mike to get a few shot examples (3+) for each of
  - Definition agent
  - Specification agent

W1    
  - Deploy on HF spaces
      - Landing Page Intro: "I am... I do... how to use me..."

W2
  - Create PDF output
  - open PDF view
  - download PDF
  - feedback box
      - good/bad
      - would you like to add/change? (Y/N)
        - if Y: what would you like to add/change:
        - if N: exit

Iteration (pitfall) - work section-by-section w/ saved states
how do you change a specific part based on feedback

### Business Value: 🚦Green Light Projects

## BizDev (v1)

Definition (What?) : Product Requirement Definition -> Specification (How?) : Engineering/Function -> Plan (When? Who?): Resource Allocator

⬇️📄 Project Output

___

imports

In [1]:
import instructor
from openai import OpenAI
import os
from pydantic import BaseModel, Field
from dotenv import load_dotenv

load_dotenv()

# Patch the OpenAI client with Instructor
client = instructor.from_openai(OpenAI(api_key=os.environ['OPENAI_API_KEY']))

User Prompt

In [2]:
user_proposal = "I want to create a portfolio chatbot that answers questions about my experience and qualifications for a role that I might be applying to"

LLM function

In [37]:
# Function
def llm_call(user_prompt, system_prompt=None, response_model=None):
    messages = [
        {"role": "user", "content": user_prompt}
    ] if system_prompt is None else [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ]
    return client.chat.completions.create(
        model="gpt-4-turbo-preview",
        response_model=response_model,
        messages=messages,
    )

# Usage:
# response = llm_call("user prompt").choices[0].message.content if response_model is None else llm_call("user prompt", response_model=response_model).choices[0].message.content

In [39]:
llm_call("testing... 1,2,3").choices[0].message.content

'Testing received loud and clear! How can I assist you today?'

### PRD Agent

In [47]:
def generate_prd_from_file(user_proposal, prd_template_file):
    # sourcery skip: inline-immediately-returned-variable
    with open(prd_template_file, 'r') as file:
        prd_template = file.read()

    prd_document = prd_template.format(user_proposal=user_proposal)
    return prd_document

# Example usage
user_proposal = "I want to create a portfolio chatbot that answers questions about my experience and qualifications for a role that I might be applying to"
prd_template_file = '../prompts/PRD_TEMPLATE.txt'  # Path to your PRD template file
# print(generate_prd_from_file(user_proposal, prd_template_file))


In [48]:
prd_sys1 = generate_prd_from_file(user_proposal, prd_template_file)
prd_response = llm_call(user_prompt=user_proposal, system_prompt=prd_sys1).choices[0].message.content
print(prd_response)

Based on your requirement for a portfolio chatbot, here's a tailored Product Requirements Document (PRD) to guide its development:

### Portfolio Chatbot PRD

#### Introduction
**Objective/Goal**:
- Develop a chatbot that interacts with potential employers or recruiters by providing detailed responses about your professional experience, qualifications, and any relevant projects or skills related to the role you're applying for.

#### Features

**Feature 1: Interactive Question-Answering** 
- **Description**: The chatbot will leverage NLP to understand and respond to queries about your professional background, skills, qualifications, and work experiences.
- **Goal**: To simulate a real-time conversation with recruiters, offering them a personalized experience.
- **Use Case**: A recruiter inquiring about your experience with Python will receive a detailed answer highlighting your years of experience, projects completed, and the proficiency level.
- **Additional Details**: The chatbot wil

In [52]:
from pydantic import BaseModel
from typing import List

class Feature(BaseModel):
    description: str
    goal: str
    use_case: str
    additional_details: str

class PRD(BaseModel):
    objective_goal: str
    features: List[Feature]
    ux_flow_design_notes: str
    system_environment_requirements: str
    assumptions: List[str]
    constraints: List[str]
    dependencies: List[str]
    prompt_engineering_practices: str
    task_composability: str
    review_approval_process: str

# Example usage with the provided PRD content
user_proposal = "I want to create a portfolio chatbot that answers questions about my experience and qualifications for a role that I might be applying to"

prd_content = PRD(
    objective_goal=f"Develop a chatbot that interacts with potential employers or recruiters by providing detailed responses about your professional experience, qualifications, and any relevant projects or skills related to the role you're applying for. User Proposal: {user_proposal}",
    features=[
        Feature(
            description="The chatbot will leverage NLP to understand and respond to queries about your professional background, skills, qualifications, and work experiences.",
            goal="To simulate a real-time conversation with recruiters, offering them a personalized experience.",
            use_case="A recruiter inquiring about your experience with Python will receive a detailed answer highlighting your years of experience, projects completed, and the proficiency level.",
            additional_details="The chatbot will not require personal user data from the recruiter, focusing solely on professional information."
        ),
        Feature(
            description="Tailor responses based on specific roles or requirements mentioned by the recruiter.",
            goal="To emphasize aspects of your portfolio that are most relevant to the position in question.",
            use_case="If a recruiter mentions looking for a frontend developer, the chatbot immediately highlights your expertise in HTML, CSS, JavaScript, and relevant projects.",
            additional_details="The system will include a mapping mechanism to correlate job roles with specific skills and experiences in your portfolio."
        )
    ],
    ux_flow_design_notes="The chatbot will be integrated on professional networking sites or personal portfolio pages with an inviting user interface prompting the recruiter to start the dialogue. Users initiate interaction by typing in a question or selecting pre-set queries about your qualifications. The chatbot will then guide the conversation based on the query, asking follow-up questions if necessary. Ensure the chat interface is accessible, intuitive, and does not require extensive technical knowledge to interact with.",
    system_environment_requirements="Supported Environments: Browsers: Chrome, Firefox, Safari. Operating Systems: Windows 10, macOS. Mobile responsiveness for access on smartphones and tablets.",
    assumptions=[
        "Recruiters have a basic understanding of how to interact with chatbots."
    ],
    constraints=[
        "The chatbot will rely on accurately updated information to remain relevant."
    ],
    dependencies=[
        "Integration with professional networking platforms for live data updates and LinkedIn APIs for fetching the latest portfolio updates."
    ],
    prompt_engineering_practices="Use structured data for the backend portfolio information to streamline clear, specific responses. Implement version control for the chatbot's knowledge base to keep it updated with the newest information. Regularly test the chatbot interaction flows and update based on feedback to improve conversational accuracy and relevance.",
    task_composability="The chatbot will handle multiple inquiries in a single session, prioritizing by the sequence of the conversation. Utilize AI conversational agents with integration of filtering tools to select the most relevant portfolio pieces for each question. Implement conditional logic to guide the flow of conversation based on the context and complexity of the inquiries.",
    review_approval_process="Initial Review: Your own review of the chatbot's responses to ensure accuracy. Business Side Stakeholders Review: Seek feedback from mentors or colleagues in your industry. Engineering Handoff: Detailed documentation for developers on the team, ensuring they understand the requirements and constraints. Final Approval: Test the chatbot with a group of recruiter contacts for real-world feedback and final adjustments."
)

print(prd_content.model_dump_json(indent=4))

{
    "objective_goal": "Develop a chatbot that interacts with potential employers or recruiters by providing detailed responses about your professional experience, qualifications, and any relevant projects or skills related to the role you're applying for. User Proposal: I want to create a portfolio chatbot that answers questions about my experience and qualifications for a role that I might be applying to",
    "features": [
        {
            "description": "The chatbot will leverage NLP to understand and respond to queries about your professional background, skills, qualifications, and work experiences.",
            "goal": "To simulate a real-time conversation with recruiters, offering them a personalized experience.",
            "use_case": "A recruiter inquiring about your experience with Python will receive a detailed answer highlighting your years of experience, projects completed, and the proficiency level.",
            "additional_details": "The chatbot will not requir

In [53]:
prd_sys1 = generate_prd_from_file(user_proposal, prd_template_file)
prd_response = llm_call(user_prompt=user_proposal, system_prompt=prd_sys1, response_model=PRD)
print(prd_response)

objective_goal='Develop a portfolio chatbot that provides detailed responses about my professional experience, skills, and qualifications to potential employers or other interested parties.' features=[Feature(description='Interactive Q&A: The chatbot engages users in a conversational manner, allowing them to inquire about specific experiences, skills, projects, or qualifications and receive comprehensive, personalized responses.', goal='Enhance user engagement by providing instant, tailored information about my professional background upon request.', use_case='An employer inquires about my project management experience, and the chatbot offers detailed examples of relevant projects, highlighting my roles and achievements.', additional_details='The chatbot should present information in a conversational tone, dynamically adjusting replies based on the context of the inquiry. It must recognize and adapt to different question types, ranging from broad overviews to specific details.'), Featu

In [54]:
print(prd_response.model_dump_json(indent=4))

{
    "objective_goal": "Develop a portfolio chatbot that provides detailed responses about my professional experience, skills, and qualifications to potential employers or other interested parties.",
    "features": [
        {
            "description": "Interactive Q&A: The chatbot engages users in a conversational manner, allowing them to inquire about specific experiences, skills, projects, or qualifications and receive comprehensive, personalized responses.",
            "goal": "Enhance user engagement by providing instant, tailored information about my professional background upon request.",
            "use_case": "An employer inquires about my project management experience, and the chatbot offers detailed examples of relevant projects, highlighting my roles and achievements.",
            "additional_details": "The chatbot should present information in a conversational tone, dynamically adjusting replies based on the context of the inquiry. It must recognize and adapt to diff